# Transformación de los datos en bruto

## Modulos a emplear

In [1]:
# Importamos las librerias a utilizar
import pandas as pd
import numpy as np
import sys
import os

# Definimos las rutas de los archivos a emplear
notebook_dir = os.getcwd()
scripts_dir = os.path.join(notebook_dir, "..", "src")
sys.path.append(scripts_dir)

# Funciones importadas como modulos
from data_transform.load_csv import load_csv
from data_transform.map_cimas_to_depth_multiple import map_cimas_create_columns

# Codigo para imprimir mas de una salida de la misma celda
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Cargamos la informacion de los registros de pozo y cimas

In [2]:
# Definimos las rutas de los registros de pozo y cimas 
logs_file_name = "Registros_C-71_P-101_P-111.csv"
tops_file_name = "Cimas_C-71_P-101_P-111 1.csv"
path_well_logs = os.path.join(notebook_dir, "..", "data", "raw", logs_file_name)
path_well_tops = os.path.join(notebook_dir, "..", "data", "raw", tops_file_name)

# Cargamos la informacion de los pozos
well_logs_df = load_csv(path_well_logs)

# Cambiamos el tipo de de columnas a float
columnas_convertir = ["md", "bound water_qe", "calcite_qe", "cgr", "dolomite_qe", "dtco", "gr", "vsh_qe",
                      "illite_qe", "nphi", "por_qe", "pore_qe", "rhob", "rp", "sw_qe", "uoil_qe", "uwater_qe"]

well_logs_df[columnas_convertir] = well_logs_df[columnas_convertir].astype(float)

# Verificamos que no exista ninguna valor negativo en md
well_logs_df = well_logs_df[well_logs_df["md"] >= 0]

# Cargamos la informacion de las cimas
well_tops_df = load_csv(path_well_tops)

# Cambiamos el tipo de columna a float
well_tops_df["md"] = well_tops_df["md"].astype(float)

# Ordena el DataFrame de cimas por 'MD' para asegurarte de que esté en el orden correcto
well_tops_df = well_tops_df.sort_values(by='md')

# Borramos la cima "PT"
well_tops_df = well_tops_df[well_tops_df["surface"] != "PT"]

## Juntamos las cimas con los pozos

In [3]:
# Mapeamos las cimas con los registros de pozo 
cpe_df = map_cimas_create_columns(well_tops_df, well_logs_df, False)

## Definimos el campo al que pertenecen los pozos

In [4]:
# Definimos el campo al que pertenecen los pozos
field = "CPE"
cpe_df["field"] = field

## Creamos columnas cretacico y jurasico

In [5]:
# Creamos la columna cretacico
cpe_df["cretacico"] = cpe_df[["mndz", "snfl", "agnv", "crtcm", "crti"]].any(axis=1)

# Creamos la columna jurasico
cpe_df["jurasico"] = cpe_df[["ttnno", "kmmgn"]].any(axis=1)

## Creamos la columna dolomia

In [6]:
# Creamos una columna donde hay presencia de dolomia
cpe_df["dolomia"] = cpe_df[["ttnno", "kmmgn"]].any(axis=1)

## Creamos el DataSet con la informacion procesada

In [7]:
# Guardamos el DataSet pijije_procesed en formato csv
processed_file_name = "cpe_eval.csv"
path_to_save = os.path.join(notebook_dir, "..", "data", "processed", processed_file_name)
cpe_df.to_csv(path_to_save, index=False)